In [1]:
import pickle
from collections import defaultdict
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical

from utils import load_embedded_data

In [2]:
# Edit to run on 10% of the data.
x_train, y_train, x_test, y_test = load_embedded_data()

# Subsample to 500 data points per task.
train_idx = pickle.load(open("output/train_idx_task_prior_small.p", "rb"))
idx = train_idx[0]
for t in range(1, 10):
    idx = np.concatenate((idx, train_idx[t]))

x_train = x_train[idx]
y_train = y_train[idx]

train_tmp = defaultdict(list)
test_tmp = defaultdict(list)

for x, l in zip(x_train, y_train):
     train_tmp[l[0]].append(x)
for x, l in zip(x_test, y_test):
     test_tmp[l[0]].append(x)

train_dict, test_dict  = {}, {}
for k, v in train_tmp.items():
     train_dict[k] = np.array(v)
for k, v in test_tmp.items():
     test_dict[k] = np.array(v)
print(len(train_dict), len(test_dict))
del train_tmp, test_tmp

X_train shape: (50000, 1000)
y_train shape: (50000, 1)
X_test shape: (10000, 1000)
y_test shape: (10000, 1)
100 100


In [3]:
# x_train, y_train, x_test, y_test = pickle.load(open('/mnt/c/Users/weiwya/Desktop/cifar_resnet50_embed.p', 'rb'))
# print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
# train_tmp = defaultdict(list)
# test_tmp = defaultdict(list)

# for x, l in zip(x_train, y_train):
#     train_tmp[l[0]].append(x)
# for x, l in zip(x_test, y_test):
#     test_tmp[l[0]].append(x)

# train_dict, test_dict  = {}, {}
# for k, v in train_tmp.items():
#     train_dict[k] = np.array(v)
# for k, v in test_tmp.items():
#     test_dict[k] = np.array(v)
# print(len(train_dict), len(test_dict))
# del train_tmp, test_tmp

In [4]:
#this function inaddition to return all data specified it can also return 
#randomly sampled data from all non specified as negative examples
def get_data(data_dict, classes, return_negative=False):
    data = []
    labels = []
    counts = 0
    for c in classes:
        size = len(data_dict[c])
        data.append (data_dict[c])
        labels += [c]*size
        counts += size
        
    
    if return_negative:
        other = []
        for k, v in data_dict.items():
            if k not in classes:
                other.append(v)
        other = np.vstack(other)
        sample_per_task =int( counts / len(classes))
        sample_idx = np.random.choice(other.shape[0], sample_per_task, replace=False)
        other = other[sample_idx]
        data.append(other)
        counts += sample_per_task
        labels+= [-1] * len(other)
    
    #shuffle data for good measure
    data = np.vstack(data)
    labels = np.array(labels)

    idx = np.random.choice(np.arange(counts), size=counts, replace=False)
    return data[idx], np.array(labels)[idx]

In [5]:
#basic MLP 
#TODO:Swap out for more  sophisticated models 

def fit_basic_model(train_data, train_labels, epochs=50, verbose=False):
    input_dim = train_data.shape[1]
    n_classes = len(np.unique(train_labels))
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(input_dim, )))
    model.add(Dropout(.25))
#     model.add(BatchNormalization())
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    model.fit(train_data, to_categorical(train_labels), 
                        epochs = epochs, 
                        verbose = verbose,
                        validation_split = 0.25,
                        shuffle=True)
    return model

    
def get_accuracy(predict, acutual):
    return np.sum(predict==acutual)/len(acutual)

In [6]:
#TODO: put into Will's PL framework

def gen_task_classes(total_classes, n_tasks=10, shuffle=False):
    class_labels = np.arange(total_classes)
    if shuffle:
        np.random.shuffle(class_labels)
    return np.array_split(class_labels, n_tasks)


def fit_all_task_models(train_dict, n_tasks=10, n_label_repeats=1,shuffle_labels=True, train_neg_exmaples=True, verbose=False):
    
    total_labels = len(train_dict)
    print('total_labels %s' %total_labels)

    labels_for_tasks = []
    for i in range(n_label_repeats):
        labels_for_tasks.append(gen_task_classes(total_labels, n_tasks, shuffle=shuffle_labels))


    labels_for_tasks = np.vstack(labels_for_tasks)
    total_tasks = len(labels_for_tasks)
    print("total number of tasks %s" %total_tasks)
                         
    transformers = {}
    task_data = {}
    task_label_lookup = {}
    voters = {}
    #this part tranins transformers for all tasks
    for task in range(total_tasks):
        actual_label_to_task_labels = {}
        task_labels_to_actual_labels = {} 
        
        task_labels = labels_for_tasks[task]        
        data, label = get_data(train_dict, task_labels, return_negative=train_neg_exmaples)
        
        if train_neg_exmaples:
            task_labels = np.append(task_labels, -1) 
        
        task_labels = np.sort(task_labels) 
        
        for i, l in enumerate(task_labels):
            actual_label_to_task_labels[l] = i
            task_labels_to_actual_labels[i] = l

        label = [actual_label_to_task_labels[x] for x in label]
        
        model = fit_basic_model(data, label, verbose=verbose)
       
        #store for later stage
        transformers[task] = model
        task_label_lookup[task] = task_labels_to_actual_labels
        task_data[task] = (data,label)
                
        print('done_transformer %s' %task)
    print('training deciders')
    
    #this part trains voter, using transformers previously trained
    for task in range(total_tasks):
        d, l = task_data[task]
        transformed = []
        #partition current task data using all  models 
        for m in transformers.values():
            mm = Model(inputs=m.inputs, outputs=m.layers[-2].output)
            transformed.append(mm.predict(d))
        transformed = np.hstack(transformed)
        voter = fit_basic_model(transformed, l, verbose=verbose)
        voters[task] = voter
        print('done_voter %s' %task)
    
    return transformers, voters, task_label_lookup, task_data


In [7]:
transformers, voters, task_label_lookup, task_data  = \
        fit_all_task_models(train_dict, n_tasks=10, n_label_repeats=5, train_neg_exmaples=True)

total_labels 100
total number of tasks 50
done_transformer 0
done_transformer 1
done_transformer 2
done_transformer 3
done_transformer 4
done_transformer 5
done_transformer 6
done_transformer 7
done_transformer 8
done_transformer 9
done_transformer 10
done_transformer 11
done_transformer 12
done_transformer 13
done_transformer 14
done_transformer 15
done_transformer 16
done_transformer 17
done_transformer 18
done_transformer 19
done_transformer 20
done_transformer 21
done_transformer 22
done_transformer 23
done_transformer 24
done_transformer 25
done_transformer 26
done_transformer 27
done_transformer 28
done_transformer 29
done_transformer 30
done_transformer 31
done_transformer 32
done_transformer 33
done_transformer 34
done_transformer 35
done_transformer 36
done_transformer 37
done_transformer 38
done_transformer 39
done_transformer 40
done_transformer 41
done_transformer 42
done_transformer 43
done_transformer 44
done_transformer 45
done_transformer 46
done_transformer 47
done_tra

In [8]:
#This is the decider part of task-unaware, 
#currenly it does argmax of sums
#TODO: exp w argmax of argmax, 
def vote_on_data(data, transforms, voters, task_label_lookup, total_classes, contains_neg=True):
    n_tasks = len(voters)
    n_classes = total_classes
    n_data = len(data)

    transformed = []
    for n in range(n_tasks):
        t = transformers[n]
        m = Model(inputs=t.inputs, outputs=t.layers[-2].output)
        transformed.append(m.predict(data))
    transformed  = np.hstack(transformed)

    all_prob = np.zeros((n_data, n_classes+1))
    for n in range(n_tasks):
        label = task_label_lookup[n]
        v = voters[n]
        vv = v.predict(transformed)
        for i, p in enumerate(vv):
            for j, pp in enumerate(p):
                all_prob[i][label[j]] += pp
    #cutout last class, because it is neg examples in every task
    return all_prob[:, :-1].argmax(axis=1)

    
    

In [9]:
#check how many times an label occurred in all tasks
occur = defaultdict(int)
for v in task_label_lookup.values():
    for vv in v.values():
        occur[vv] += 1
        
total_correct = 0
for c_class in range(100):
    data = np.array(test_dict[c_class])
    predict = vote_on_data(data, transformers, voters, task_label_lookup, 100, contains_neg=False)
    total_correct += np.sum( predict == [c_class]*len(data)) 
    print(c_class,  occur[c_class], get_accuracy(predict, [c_class]*len(data)))

print(total_correct / (100 * 100))

0 5 0.51
1 5 0.61
2 5 0.36
3 5 0.38
4 5 0.18
5 5 0.07
6 5 0.27
7 5 0.54
8 5 0.38
9 5 0.58
10 5 0.17
11 5 0.15
12 5 0.44
13 5 0.09
14 5 0.13
15 5 0.55
16 5 0.68
17 5 0.31
18 5 0.36
19 5 0.51
20 5 0.5
21 5 0.66
22 5 0.61
23 5 0.47
24 5 0.07
25 5 0.13
26 5 0.33
27 5 0.15
28 5 0.45
29 5 0.36
30 5 0.16
31 5 0.52
32 5 0.33
33 5 0.26
34 5 0.59
35 5 0.08
36 5 0.53
37 5 0.3
38 5 0.36
39 5 0.62
40 5 0.48
41 5 0.62
42 5 0.46
43 5 0.7
44 5 0.35
45 5 0.14
46 5 0.18
47 5 0.13
48 5 0.32
49 5 0.55
50 5 0.06
51 5 0.19
52 5 0.39
53 5 0.57
54 5 0.38
55 5 0.19
56 5 0.27
57 5 0.41
58 5 0.41
59 5 0.27
60 5 0.52
61 5 0.43
62 5 0.16
63 5 0.2
64 5 0.16
65 5 0.14
66 5 0.33
67 5 0.16
68 5 0.44
69 5 0.54
70 5 0.18
71 5 0.28
72 5 0.16
73 5 0.31
74 5 0.1
75 5 0.65
76 5 0.4
77 5 0.45
78 5 0.57
79 5 0.58
80 5 0.48
81 5 0.46
82 5 0.65
83 5 0.51
84 5 0.31
85 5 0.38
86 5 0.54
87 5 0.59
88 5 0.49
89 5 0.65
90 5 0.06
91 5 0.32
92 5 0.28
93 5 0.12
94 5 0.64
95 5 0.57
96 5 0.19
97 5 0.49
98 5 0.32
99 5 0.49
0.3712
